In [1]:
# Import library yang dibutuhkan
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

# Models adalah kumpulan class yang digunakan untuk data training, berbentuk 2D array
models = [[GaussianNB(), 'GNB'],
          [KNeighborsClassifier(), 'KNN'],
          [SVC(), 'SVM'],
          [DecisionTreeClassifier(), 'Decision Tree']]

# data adalah csv train.csv yang digunakan untuk training, dengan syarat train.csv di download dulu dari kaggle Spaceship Titanic Problem dan dimasukkan ke folder yang sama dengan file ini
data = pd.read_csv('train.csv')

# test adalah data test yang digunakan untuk membuat output berupa csv untuk disubmit, dengan syarat test.csv di download dulu dari kaggle Spaceship Titanic Problem dan dimasukkan ke folder yang sama dengan file ini
test = pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
# Melihat informasi data
data.head()

In [ ]:
# Melihat informasi data 2
data.info()

In [ ]:
# Fitur yang digunakan adalah berupa array, digunakan untuk menyeleksi fitur yang relevan untuk prediksi
fitur_yang_digunakan = ['HomePlanet', 'CryoSleep', 'VIP', 'Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Target adalah data yang ingin diprediksi
target = ['Transported']

In [ ]:
# Imputer digunakan untuk mengisi value yang bersifat NaN atau kosong, agar data bisa di train
imputer = SimpleImputer(strategy='most_frequent')

# Proses imputasi data dari NaN ke data yang sering muncul di fitur tersebut
data.iloc[:, :-1] = imputer.fit_transform(data.iloc[:, :-1])
test.iloc[:, :] = imputer.transform(test.iloc[:, :])

In [ ]:
# Proses seleksi fitur yang dipakai, dari yang lengkap, menjadi fitur-fitur yang relevan saja yang dipakai
data = data[fitur_yang_digunakan + target]
test = test[fitur_yang_digunakan]

# Untuk mengetahui jumlah fitur yang terlibat, bisa print n untuk mengetahui jumlah fitur yang dipakai
fitur = data.drop(columns=target)
n = fitur.shape[1]

# print(n)

# Mulai memisahkan antara fitur dengan target
arr = data.values
X = arr[:,0:n]
y = arr[:,n]

In [ ]:
# Label encoder digunakan untuk mengubah data berupa kategori menjadi numerik, misal data berupa low, medium, high akan diganti dengan angka misal 0, 1, 2 agar dapat di train
encoder = LabelEncoder()

# Untuk setiap kolom yang berupa kategori, dibuat menjadi bentuk numerik
for kolom in fitur.select_dtypes(include='object'):
  fitur[kolom] = encoder.fit_transform(fitur[kolom])
  test[kolom] = encoder.transform(test[kolom])

# Dimasukkan dalam variabel X dan X_test, X untuk data training, dan X_test untuk data lengkapnya
X = fitur.values
X_test = test.values

In [ ]:
# Mengecek apakah X sudah berupa numerik semua
X

In [ ]:
# Mengecek apakah X_test sudah berupa numerik semua
X_test

In [ ]:
# Mengecek bentuk dari kolom target yaitu y
y

In [ ]:
# Mengubah bentuk y menjadi numerik agar dapat mengambil classification reportnya
y = np.array(y, dtype=int)

In [ ]:
# Mengecek jika y sudah berbentuk numerik
y

In [ ]:
# X dan y diambil dari data training, sehingga kita membuat data test sendiri untuk proses training, atau yang disebut dengan X_validation dan y_validation menggunakan library sklearn.model_selection train_test_split
# test_size 0.2 berarti 80% data merupakan X_train dan y_train, 20% data merupakan X_validation dan y_validation
# random_state hanya agar train_test_split bersifat konsisten atau tidak berubah-ubah, bisa diisi nilai bilangan bulat terserah
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)

In [ ]:
# Fungsi ini diambil dari modul 8 Praktikum
results = []

for model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model[0], X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    print('%s: %f (%f)' % (model[1], cv_results.mean(), cv_results.std()))

# Karena SVM adalah model yang memiliki akurasi paling tinggi, maka kita akan menggunakan SVM untuk data training

In [ ]:
# Verifikasi ulang data, apakah data berubah, seharusnya data tidak berubah karena hanya mengambil value dari data (variabel arr di cell 23)
data.head()

In [ ]:
# Proses training data menggunakan model terbaik, memanggil fungsi fit
models[2][0].fit(X, y)

# Prediksi data dari data train 20%
prediksi = models[2][0].predict(X_validation)

# Print report klasifikasi
print("Report klasifikasi untuk model terbaik yaitu SVM: ")
print(classification_report(y_validation, prediksi))

In [ ]:
# Prediksi data dari data test dari test.csv yang bentuknya sudah diubah agar dapat diprediksi berdasarkan data training
prediksi = models[2][0].predict(X_test)

# Karena prediksi berupa angka, harus diubah kembali dalam bentuk True dan False
label = {0: "False", 1: "True"}
prediksi_text = [label[val] for val in prediksi]

# Mengambil PassengerId dari test.csv, karena test.csv sebelumnya sudah ditransform agar bisa diprediksi
id_penumpang = pd.read_csv('test.csv')['PassengerId']

# Membuat panda dataframe dari PassengerId dan hasil prediksi yang sudah dibuat
output_df = pd.DataFrame({'PassengerId': id_penumpang, 'Transported': prediksi_text})

# Output csv, kalau di run harusnya keluar prediksi.csv didalam folder, submit file tersebut ke kaggle Spaceship Titanic Problem
output_df.to_csv("prediksi.csv", index=False)